In [53]:
import requests
import re
import pandas as pd
import json
import time
import numpy as np

# requires pip install sentence-transformers==2.2.2
from sentence_transformers import SentenceTransformer

In [16]:
def get_reddit(subreddit, after=None):
    time.sleep(0.5)    
    source_file = f'reddit-{subreddit}.txt'
    base_url = f'https://www.reddit.com/r/{subreddit}/controversial.json?limit=100&t=year'
    
    if after is not None:
        base_url += f'&after=t3_{after}'
        source_file = f'reddit-{subreddit}_{after}.txt'        
        
    print(f'Loading posts from {base_url}')        
    #request = requests.get(base_url, headers = {'User-agent': 'development'})
    #response = request.json()
    with open(source_file, "r") as infile:
        return json.load(infile)

In [18]:
raw_posts = {}
for subreddit in ['bullying', 'selfimprovement', 'depression', 'FengShui']:
    raw_posts[subreddit] = []
    last_identifier = None
    for _ in range(0, 10):
        raw_response = get_reddit(subreddit, after=last_identifier)

        for post in raw_response['data']['children']:
                raw_posts[subreddit].append(post)            
                last_identifier = post['data']['id']

Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_w9924g
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_zagcku
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_ycqpjc
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_xf1ifr
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_wbzt8u
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_vd11ym
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_ufos2a
Loading posts from https://www.reddit.com/r/bullying/controversial.json?limit=100&t=year&after=t3_ta46qm
Loading posts from https://www.reddit.com/r/bullying/controversial.json

In [24]:
subreddits = ['bullying', 'selfimprovement', 'depression', 'FengShui']

prepared_sentences = []
for subreddit in subreddits:
    for num, post in enumerate(raw_posts[subreddit]):
        identifier = post['data']['id']
        text = post['data']['selftext']
        sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
        for index, sentence in enumerate(sentences):
            sentence_id = identifier + '_' + str(index)
            prepared_sentences.append((sentence_id, sentence, subreddit))
            
labeled_sentences = pd.DataFrame.from_records(prepared_sentences, columns=['id', 'text', 'origin'])            

In [25]:
labeled_sentences

,id,text,origin
0,x6d5qc_0,I was bullied by girls all my life but I would...,bullying
1,x6d5qc_1,They can't even bully rightly like verbally as...,bullying
2,x6d5qc_2,Nothing\nThey can't even bully rightly \nAlso ...,bullying
3,x6d5qc_3,\nAlso don't watch a woman's cunt only a man's...,bullying
4,x6d5qc_4,,bullying
...,...,...,...
39796,rpabe4_8,width=3371&amp;format=png&amp;auto=webp&amp;s=...,FengShui
39797,rp6j88_0,Just wondering if anyone else is doing a deep...,FengShui
39798,rp6j88_1,I got into the cleaning spirit on bedroom clos...,FengShui
39799,rp6j88_2,"Planning on painting by spring , so cleaning n...",FengShui


In [29]:
url_regex = '((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
filtered_sentences = labeled_sentences[labeled_sentences.text.str.len() > 20]
filtered_sentences = filtered_sentences[~filtered_sentences.text.str.match(url_regex)]

In [30]:
filtered_sentences

,id,text,origin
0,x6d5qc_0,I was bullied by girls all my life but I would...,bullying
1,x6d5qc_1,They can't even bully rightly like verbally as...,bullying
2,x6d5qc_2,Nothing\nThey can't even bully rightly \nAlso ...,bullying
3,x6d5qc_3,\nAlso don't watch a woman's cunt only a man's...,bullying
5,vcbknn_0,"""Bullying is coercion--Implicit or explicit th...",bullying
...,...,...,...
39790,rpabe4_2,"In other words, all the other houses on that s...",FengShui
39796,rpabe4_8,width=3371&amp;format=png&amp;auto=webp&amp;s=...,FengShui
39797,rp6j88_0,Just wondering if anyone else is doing a deep...,FengShui
39798,rp6j88_1,I got into the cleaning spirit on bedroom clos...,FengShui


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer

train_data, test_data = train_test_split(filtered_sentences, test_size=0.2, random_state=42, shuffle=True)

# There are better, more expensive models: https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('all-MiniLM-L6-v2')

encoder = FunctionTransformer(lambda row: model.encode(row['text'].values))

In [42]:
X_train = encoder.transform(train_data)

In [43]:
X_test = encoder.transform(test_data)

In [62]:
from tensorflow.keras.layers import Dense  # pylint: disable=no-name-in-module
from tensorflow.keras.models import Sequential  # pylint: disable=no-name-in-module
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD  # pylint: disable=no-name-in-module
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier  # pylint: disable=no-name-in-module

def create_model(input_dim=384):
    clf = Sequential()
    clf.add(Dense(16, activation='relu', input_dim=input_dim))
    clf.add(Dense(8, activation='relu'))
    clf.add(Dense(2, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=["accuracy"])
    return clf

neuralnet = KerasClassifier(build_fn=create_model, epochs=20, batch_size=64, verbose=1)

In [67]:
y_train = np.array(train_data.origin.isin(['bullying', 'depression']))
y_test = np.array(test_data.origin.isin(['bullying', 'depression']))

In [68]:
neuralnet.fit(X_train, y_train)

Epoch 1/20
398/398 [==============================] - 0s 638us/step - loss: 0.6672 - accuracy: 0.6195
Epoch 2/20
398/398 [==============================] - 0s 640us/step - loss: 0.6600 - accuracy: 0.6206
Epoch 3/20
398/398 [==============================] - 0s 530us/step - loss: 0.6563 - accuracy: 0.6206
Epoch 4/20
398/398 [==============================] - 0s 600us/step - loss: 0.6508 - accuracy: 0.6206
Epoch 5/20
398/398 [==============================] - 0s 582us/step - loss: 0.6423 - accuracy: 0.6206
Epoch 6/20
398/398 [==============================] - 0s 592us/step - loss: 0.6300 - accuracy: 0.6206
Epoch 7/20
398/398 [==============================] - 0s 547us/step - loss: 0.6138 - accuracy: 0.6277
Epoch 8/20
398/398 [==============================] - 0s 555us/step - loss: 0.5954 - accuracy: 0.6688
Epoch 9/20
398/398 [==============================] - 0s 540us/step - loss: 0.5775 - accuracy: 0.6998
Epoch 10/20
398/398 [==============================] - 0s 550us/step - loss: 0.562

In [69]:
neuralnet.score(X_test, y_test)

100/100 [==============================] - 0s 970us/step - loss: 0.5139 - accuracy: 0.7461


0.7461115717887878